In [1]:
using Rx
using BenchmarkTools

┌ Info: Precompiling Rx [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1273


In [2]:
function standart_approach(source)
    output = Vector{Float64}()
    for value in source
        if value % 17 == 0
            push!(output, sqrt(value))
        end
    end
    return output
end

standart_approach (generic function with 1 method)

In [3]:
@CreateFilterOperator(Div17, Int, (d) -> d % 17 == 0)
@CreateMapOperator(Sqrt, Int, Float64, (d) -> sqrt(d))

In [4]:
mutable struct KeepLastActor{D} <: NextActor{D}
    value :: Union{Nothing, D}
    
    KeepLastActor{D}() where D = new(nothing)
end

Rx.on_next!(actor::KeepLastActor{D}, data::D) where D = actor.value = data

struct KeepActor{D} <: NextActor{D}
    values :: Vector{D}
    
    KeepActor{D}() where D = new(Vector{D}())
end

Rx.on_next!(actor::KeepActor{D}, data::D) where D = push!(actor.values, data)

In [5]:
function reactive_approach(source)
    actor = KeepActor{Float64}()
    subscribe!(source |> Div17FilterOperator() |>  SqrtMapOperator(), actor)
    return actor.values
end

reactive_approach (generic function with 1 method)

In [6]:
standart_approach([i for i in 1:100]) == reactive_approach(from([i for i in 1:100]))

true

In [7]:
n = 1000000

source1 = [i for i in 1:n]
source2 = from([i for i in 1:n])

@btime standart_approach(source1);
@btime reactive_approach(source2);

  1.531 ms (16 allocations: 1.00 MiB)
  2.341 ms (21 allocations: 1.00 MiB)
